In [ ]:
"""" Contains Progressive Neural Network Class"""

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy.io import loadmat

In [ ]:
BATCH_SIZE = 500
NUM_SGD_STEPS = 3000

In [ ]:
class SingleTaskNetwork:
    """
    Creates a NN for single task

    """
    def __init__(self, size, activations, input_ph, col_num, dropout=False, batch_norm=False):
        """
        Parameters
        ----------
        size: list of ints
            list of units in hidden layers and output layer 
            The last element should be the output size, example: [128,128,1]
        activations: list of tf.nn activations
            list of activation functions for each layer, 
            Size of the list should be number of hidden layers + 1, 
            the last element should be the output activation function, example: [tf.nn.relu,tf.nn.relu, None]
        input_ph: tf placeholder
            TensorFlow placeholder for the inputs
        dropout: boolean, optional
            specifies whether to use dropout
        batch_norm: boolean, optional
            specifies whether to use batch normalization
        """
        self.size = size
        self.activations = activations
        self.input_ph = input_ph
        self.dropout = dropout
        self.batch_norm = batch_norm
        self.weights = []
        self.biases = []
        self.h = [self.input_ph]
        self.col_num = col_num

    def create_single_task_nn(self):
        """ Creates a the neural network for regression problem

        Returns
        ----------
        output_layer: Output layer prediction
        """
        

        last_layer = self.input_ph

        scope = "nn"
        if self.dropout:
            scope += "_d"
        if self.batch_norm:
            scope += "bn"
            
        # create variables
        for layer in range(len(self.size)):
            if layer == 0:  # input layer                
                self.weights.append(tf.get_variable(
                    name='W{}_{}'.format(layer, self.col_num),
                    shape=[self.input_ph.shape[1], self.size[0]], 
                    initializer=tf.contrib.layers.xavier_initializer()
                ))
                self.biases.append(tf.get_variable(
                    name='b{}_{}'.format(layer, self.col_num), 
                    shape=[self.size[0]], 
                    initializer=tf.constant_initializer(0.)
                ))
            else:   # hidden and output layers:
                shape = self.size[layer-1:layer+1]
                self.weights.append(tf.get_variable(
                    name='W{}_{}'.format(layer, self.col_num), 
                    shape=shape, 
                    initializer=tf.contrib.layers.xavier_initializer()
                ))
                self.biases.append(tf.get_variable(
                    name='b{}_{}'.format(layer, self.col_num), 
                    shape=shape[1], 
                    initializer=tf.constant_initializer(0.)
                ))

        # create computation graph
        i = 0
        for W, b, activation in zip(self.weights, self.biases, self.activations):
            last_layer = tf.matmul(last_layer, W) + b
            self.h.append(last_layer)
            if activation is not None:
                last_layer = activation(last_layer)
            if self.dropout:
                last_layer = tf.nn.dropout(last_layer, 0.5)
            if self.batch_norm:
                last_layer = tf.contrib.layers.batch_norm(last_layer, 
                                                          center=True, scale=True, 
                                                          scope="{}_{}".format(scope,i), 
                                                          reuse=tf.AUTO_REUSE)
            i += 1
        self.output_pred = last_layer        
        
        return self.output_pred

In [ ]:
class ProgressiveNN:
    """
    Creates a progressive neural network. The last column will be the target task column.
    Other columns represent source tasks

    """
    def __init__(self, size, activations, input_ph, num_source_cols, dropout=False, batch_norm=False):
        """
        Parameters
        ----------
        size: list of ints
            list of units in hidden layers and output layer 
            The last element should be the output size, example: [128,128,1]
        activations: list of tf.nn activations
            list of activation functions for each layer, 
            Size of the list should be number of hidden layers + 1, 
            the last element should be the output activation function, example: [tf.nn.relu,tf.nn.relu, None]
        input_ph: tf placeholder
            TensorFlow placeholder for the inputs
        num_source_cols: int, 
            Number of source task columns. 
        dropout: boolean, optional
            specifies whether to use dropout
        batch_norm: boolean, optional
            specifies whether to use batch normalization
        """
        self.size = size
        self.activations = activations
        self.input_ph = input_ph
        self.num_source_cols = num_source_cols
        self.dropout = dropout
        self.batch_norm = batch_norm
        self.last_layer = self.input_ph
        self.num_layers = len(self.size)
        self.num_cols = self.num_source_cols + 1  # 1 is the target domain net
        self.source_weights = [[] for _ in range(self.num_source_cols)] 
        self.source_biases = [[] for _ in range(self.num_source_cols)]
        self.latteral_connectins = []
        self.latteral_conns = []
        for l in range(self.num_layers):
            self.latteral_conns.append( [[]] * self.num_cols )
        self.latteral_conns[0] = None
        self.target_weights = []
        self.target_biases = []
        

    def create_progressive_nn(self):
        """ Creates a the neural network for regression problem

        Returns
        ----------
        output_layer: Output layer prediction
        """
        scope = "nn"
        if self.dropout:
            scope += "_d"
        if self.batch_norm:
            scope += "bn"
   
        # create weights and biases
        col_objs = [SingleTaskNetwork(self.size, self.activations, self.input_ph, col)
                    for col in range(self.num_cols)]
        for col_obj in col_objs:
            col_obj.create_single_task_nn()
            
        # create connections and computation graph
        if self.num_cols == 1:
            return col_objs[0].output_pred
        
        last_layer = [self.input_ph for _ in range(self.num_cols)] 
      
        for layer in range(self.num_layers):   
            for col in range(self.num_cols): 
                last_layer[col] = tf.matmul(last_layer[col], col_objs[col].weights[layer]) + col_objs[col].biases[layer]
                if col > 0 and layer > 0:
                    for c in range(col):
                        U_shape = [col_objs[c].size[layer-1], col_objs[col].size[layer]]  # from layer l-1 of all prev columns
                        new_U = tf.get_variable(name='U_{}_{}_{}'.format(layer,col,c),   
                                                shape=U_shape, 
                                                initializer=tf.contrib.layers.xavier_initializer())
                        self.latteral_conns[layer][col].append(new_U)
                        last_layer[col] += tf.matmul(col_objs[c].h[layer-1],new_U)
                        
                if col_objs[col].activations[layer] is not None:
                    last_layer[col] = col_objs[col].activations[layer](last_layer[col])
                if self.dropout:
                    last_layer[col] = tf.nn.dropout(last_layer[col], 0.5)
                if self.batch_norm:
                    last_layer[col] = tf.contrib.layers.batch_norm(last_layer[col], 
                                                              center=True, scale=True, 
                                                              scope="{}_{}".format(scope,i), 
                                                              reuse=tf.AUTO_REUSE)
                col_objs[col].h[layer] = last_layer[col]
        output_pred = last_layer[col]        
        
        return output_pred


In [ ]:
###########################################################

tf.reset_default_graph()

# state and action placeholders
output_ph = tf.placeholder(dtype=tf.float32, shape=[None, 1])
input_ph = tf.placeholder(dtype=tf.float32, shape=[None, train_set.shape[1]-2])

# session
sess = tf.Session()

# function approximator
# func_approx = create_nn(input_ph, dropout=True)
size = [128,1]
activations = [tf.nn.leaky_relu,None]
num_source_cols = 1
obj = ProgressiveNN(size, activations, input_ph, num_source_cols)
func_approx = obj.create_progressive_nn()
# loss
loss = tf.reduce_mean(0.5 * tf.square(output_ph - func_approx))


learning_rate = [0.01]  #[12e-3, 11e-3, 9e-3, 0.01]

res_dropout = []
min_error = float("inf")

for i in learning_rate:  # grid search
    res_dropout.append([])
    # optimizer
    opt = tf.train.AdamOptimizer(learning_rate= i).minimize(loss)

    # initialize all the variables
    sess.run(tf.global_variables_initializer())

    for j in range(NUM_SGD_STEPS):  # train for 1000 SGD steps
        # collect a batch of data
        indices = np.random.randint(low=0, 
                                    high=train_set.shape[0],
                                    size=BATCH_SIZE)
        obs = train_set[indices,1:-1]
        actions = train_set[indices,-1:]

        # compute the loss/return and update the weights of the policy
        _ = sess.run(opt, feed_dict={input_ph: obs, output_ph: actions})

        # compute the loss on the test set
        mse = sess.run(loss, feed_dict={input_ph: test_set[:,1:-1],
                                        output_ph: test_set[:,-1:]})

        res_dropout[-1].append(mse)
        
        if mse < min_error:
            min_error = mse
            best_learning_rate = i
            
        print(i, j, mse)
print ("best_learning_rate: %s" % best_learning_rate)